In [9]:
import tensorflow as tf
import numpy as np
import os, random
from pathlib import Path
from tensorflow.keras.preprocessing import image

from tensorflow.keras.applications.efficientnet import preprocess_input

In [ ]:
def load_and_preprocess(img_path, img_size=300):
    img = image.load_img(img_path, target_size=(img_size, img_size))
    arr = image.img_to_array(img)
    arr = preprocess_input(arr)
    return tf.expand_dims(arr, 0)

In [11]:
def evaluate_random_samples(model_path, val_dir, k=100, img_size=300):
    """
    随机抽取 k 张验证集图片，汇总每个真实类别被预测成各类别的计数，
    并打印每类样本数、每类正确率、误判去向。
    """
    # 1) 取得类别顺序
    ds = tf.keras.preprocessing.image_dataset_from_directory(
        os.path.join(val_dir, '..', 'train'),
        image_size=(img_size, img_size), batch_size=32, shuffle=False)
    class_names = ds.class_names
    name2label  = {n:i for i,n in enumerate(class_names)}
    n_classes   = len(class_names)

    # 2) 收集验证集中所有图片(路径, 真实标签)
    all_imgs = []
    for cls in class_names:
        for p in (Path(val_dir)/cls).glob('*'):
            if p.suffix.lower() in ['.jpg','.jpeg','.png','.bmp','.gif']:
                all_imgs.append((str(p), name2label[cls]))

    random.shuffle(all_imgs)
    samples = all_imgs[:k]

    # 3) 预测
    model = tf.keras.models.load_model(model_path)
    conf_mat = np.zeros((n_classes, n_classes), dtype=int)   # [真实, 预测]

    for p, true_lab in samples:
        pred = np.argmax(model.predict(load_and_preprocess(p, img_size))[0])
        conf_mat[true_lab, pred] += 1

    # 4) 汇总 & 打印
    print(f"🎯 随机抽取 {k} 张验证集图片后的统计：\n")
    for i, cls in enumerate(class_names):
        row = conf_mat[i]
        total   = row.sum()
        correct = row[i]
        if total == 0:
            print(f"- {cls:<15}: 样本 0")
            continue
        acc = correct / total
        # 构造“被预测成 X 的张数”摘要
        miss = {class_names[j]:row[j] for j in range(n_classes) if j!=i and row[j]>0}
        miss_str = ", ".join([f"{n}:{c}" for n,c in miss.items()]) if miss else "—"
        print(f"- {cls:<15}: 样本 {total:<3}  正确 {correct:<3}  准确率 {acc:6.1%}  误判→ {miss_str}")

    overall_acc = np.trace(conf_mat) / k
    print(f"\n✅ 总体准确率：{overall_acc:.2%}")
    return conf_mat


In [12]:
# ----------- 调用示例 -----------
model_path = 'final_model_full.h5'               # 你的模型
val_dir    = 'DL_Classify/validation'            # validation 根目录
k          = 30                                  # 想抽多少张

evaluate_random_samples(model_path, val_dir, k)


Found 1306 files belonging to 5 classes.


2025-07-03 21:26:35.657426: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 25ms/step
🎯 随机抽取 30 张验证集图片后的统计：

- Pallas cats    : 样本 7    正确 7    准确率 100.0%  误判→ —
- Persian cats   : 样本 6    正确 6    准确率 100.0%  误判→ —
- Ragdolls       : 样本 6    正确 6    准确率 100.0%  误判→ —
- Singapura cats : 样本 7    正确 7    准确率 100.0%  误判→ —
- Sphynx cats    : 样本 4    正确 4    准确率 100.0%  误判→ —

✅ 总体准确率：100.00%


array([[7, 0, 0, 0, 0],
       [0, 6, 0, 0, 0],
       [0, 0, 6, 0, 0],
       [0, 0, 0, 7, 0],
       [0, 0, 0, 0, 4]])